In [1]:
import sys
sys.path.append('Scripts')
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np

from utils import path
from DataVis import createInputFigure
from NNUtils import *
from NNCreationAux import *

from sklearn import preprocessing
import os
import time
import copy

***

In [ ]:
modelType = 'resnet'
modelSize = '101'
data_type = '512'
figSize = '128'
folder = 'GAF_Combinados_difference'

model_name = modelType+modelSize

In [ ]:
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder)

In [ ]:
model = model_selection(modelName, numClasses)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)
model, stats = train(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs = 2)

***
Example

In [10]:
model = models.resnet18(pretrained = True)
model.fc = nn.Linear(512, 5)

In [ ]:
def imshow(inp, title = None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    inp = std*inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize = (25, 25))
    plt.axis("off")
    plt.title("Training Images")
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated
    
# Get a batch of training data
dataloaders, dataset_sizes, dataloader, class_names = getData('512')
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from a batch
out = utils.make_grid(inputs)

imshow(out, title = [class_names[x] for x in classes])

***

In [2]:
path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\GAF_Combinados_summation\\512'
dataset = get_dataset(path, 224)
data_type = '512'
folder = 'GAF_Combinados_difference'
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder, batch_size = 8)

In [3]:
# model = models.resnet101(pretrained = True)
# model = model.to(device)
res18 = model_selection('resnet18', 5).to(device)
optimizer = optim.SGD(res18.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [15]:
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle = False)
temp = enumerate(kfold.split(dataset, dataset.targets))


In [16]:
fold, (train_index, test_index) = list(temp)[0]

In [22]:
train_subset = torch.utils.data.Subset(dataset, train_index)
test_subset = torch.utils.data.Subset(dataset, test_index)


dataset_size = {}
dataset_size['train'] = len(train_subset) 
dataset_size['test'] = len(test_subset)
dataloaders  = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_subset, batch_size = 4, shuffle = False)
dataloaders['test']  = torch.utils.data.DataLoader(test_subset, batch_size = 4, shuffle = False)

In [28]:
res18, *stats = train(res18, dataloaders, dataset_size, criterion, optimizer, scheduler, num_epochs = 10)

--
Epoch 1/10
-----
Stage: train
Loss: 0.7753 - Accuracy: 0.8810
Stage: test
Loss: 28.8597 - Accuracy: 0.0000
--
Epoch 2/10
-----
Stage: train


KeyboardInterrupt: 

In [6]:
res18, best_acc, mean_acc = kfold_train(res18, dataset, criterion, optimizer, scheduler)

Fold 0
--
Epoch 1/10
-----
Stage: train
Loss: 0.8409 - Accuracy: 0.9487
Stage: test
Loss: 32.8829 - Accuracy: 0.0000
--
Epoch 2/10
-----
Stage: train
Loss: 1.7678 - Accuracy: 0.8672
Stage: test
Loss: 86.0056 - Accuracy: 0.0000
--
Epoch 3/10
-----
Stage: train
Loss: 1.0496 - Accuracy: 0.8727
Stage: test
Loss: 35.3036 - Accuracy: 0.0000
--
Epoch 4/10
-----
Stage: train


KeyboardInterrupt: 

In [5]:
res18, *stats = train(res18, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs = 10)

--
Epoch 1/10
-----
Stage: train
Loss: 1.5149 - Accuracy: 0.3558
Stage: test
Loss: 1.2196 - Accuracy: 0.5055
--
Epoch 2/10
-----
Stage: train
Loss: 0.8934 - Accuracy: 0.6690
Stage: test
Loss: 1.0112 - Accuracy: 0.6282
--
Epoch 3/10
-----
Stage: train
Loss: 0.4626 - Accuracy: 0.8352
Stage: test
Loss: 0.7981 - Accuracy: 0.7509
--
Epoch 4/10
-----
Stage: train
Loss: 0.3623 - Accuracy: 0.8750
Stage: test
Loss: 0.7742 - Accuracy: 0.7601
--
Epoch 5/10
-----
Stage: train
Loss: 0.3015 - Accuracy: 0.8965
Stage: test
Loss: 0.7236 - Accuracy: 0.7875
--
Epoch 6/10
-----
Stage: train
Loss: 0.2647 - Accuracy: 0.9080
Stage: test
Loss: 0.6876 - Accuracy: 0.8223
--
Epoch 7/10
-----
Stage: train
Loss: 0.2203 - Accuracy: 0.9263
Stage: test
Loss: 0.5855 - Accuracy: 0.8443
--
Epoch 8/10
-----
Stage: train
Loss: 0.1181 - Accuracy: 0.9574
Stage: test
Loss: 0.4505 - Accuracy: 0.8791
--
Epoch 9/10
-----
Stage: train
Loss: 0.0717 - Accuracy: 0.9771
Stage: test
Loss: 0.4439 - Accuracy: 0.8773
--
Epoch 10/10
----

In [6]:
# model = models.resnet101(pretrained = True)
# model = model.to(device)
res34 = model_selection('resnet34', 5).to(device)
optimizer = optim.SGD(res34.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [7]:
res34, *stats = train(res34, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs = 10)

--
Epoch 1/10
-----
Stage: train
Loss: 1.6225 - Accuracy: 0.3136
Stage: test
Loss: 1.4783 - Accuracy: 0.4103
--
Epoch 2/10
-----
Stage: train
Loss: 1.0813 - Accuracy: 0.5861
Stage: test
Loss: 1.1419 - Accuracy: 0.5806
--
Epoch 3/10
-----
Stage: train
Loss: 0.6423 - Accuracy: 0.7647
Stage: test
Loss: 0.8425 - Accuracy: 0.7216
--
Epoch 4/10
-----
Stage: train
Loss: 0.4317 - Accuracy: 0.8457
Stage: test
Loss: 0.9156 - Accuracy: 0.7216
--
Epoch 5/10
-----
Stage: train
Loss: 0.3991 - Accuracy: 0.8585
Stage: test
Loss: 0.6508 - Accuracy: 0.7784
--
Epoch 6/10
-----
Stage: train
Loss: 0.1918 - Accuracy: 0.9286
Stage: test
Loss: 0.6303 - Accuracy: 0.8333
--
Epoch 7/10
-----
Stage: train
Loss: 0.1667 - Accuracy: 0.9423
Stage: test
Loss: 0.5953 - Accuracy: 0.8278
--
Epoch 8/10
-----
Stage: train
Loss: 0.1057 - Accuracy: 0.9629
Stage: test
Loss: 0.5020 - Accuracy: 0.8553
--
Epoch 9/10
-----
Stage: train
Loss: 0.0500 - Accuracy: 0.9863
Stage: test
Loss: 0.4637 - Accuracy: 0.8626
--
Epoch 10/10
----

***

In [10]:
batch_size = 1
image_size = 224
folder = 'Test_GASF'
figPath = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\'+folder+'\\512'
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder, batch_size = 1)

In [13]:
class predictor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def predict(self, dataloader):
        outputs = []
        targets = []
        for sample, target in dataloader:
            sample = sample.to(device)
            target = target.to(device)
            o = torch.argmax(F.softmax(self.model(sample), dim = 1)).item()
            outputs.append(o)
            targets.append(target.item())
            
        outputs = np.array(outputs)
        targets = np.array(targets)
        right_pred = 1*(outputs == targets)
        return outputs, right_pred

In [14]:
predic = predictor(res18)
_, outres18 = predic.predict(dataloader)

In [15]:
predic = predictor(res34)
_, outres34 = predic.predict(dataloader)

In [19]:
np.count_nonzero(outres18)/500

0.178

In [20]:
np.count_nonzero(outres34)/500

0.202

In [22]:
from scipy import stats
stats.ttest_ind(outres18, outres34, equal_var=True)

Ttest_indResult(statistic=-0.9667862845385763, pvalue=0.33388509993212834)